In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
!unzip /content/drive/MyDrive/data/large.zip

Data Split

In [ ]:
train_dir = '/content/drive/MyDrive/data/split_data/train'
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=16,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))

val_dir = '/content/drive/MyDrive/data/split_data/validation'
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator =train_datagen.flow_from_directory(val_dir,
                                                    batch_size=16,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))

Found 806 images belonging to 3 classes.
Found 102 images belonging to 3 classes.


In [ ]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation="softmax")
])

model.summary()

# model.compile(optimizer=RMSprop(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['acc'])
model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=['acc'])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
history = model.fit(train_generator,
                    epochs = 30,
                    callbacks=[early_stopping],
                    validation_data = validation_generator)

In [ ]:
model.save('/content/drive/MyDrive/data/cnn-model1.keras')

In [ ]:
test_dir = '/content/large/test'
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  batch_size=16,
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  target_size=(150, 150))

In [ ]:
print('Test Confusion Matrix:')
print(confusion_matrix(y_test, y_test_pred))
print('Test Classification Report:')
print(classification_report(y_test, y_test_pred, target_names=classes))

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(nrows=2, ncols=5, figsize=(20, 10))  # Create 2 rows and 5 columns of subplots
idx = 0
labels = list(test_generator.class_indices.keys())  # Get the class labels from the generator
for i in range(2):
    for j in range(5):
        predicted_label = labels[np.argmax(predictions[idx])]  # Get the predicted label
        ax[i, j].imshow(test_generator[0][0][idx])  # Display the image
        ax[i, j].set_title(f"Predicted: {predicted_label}")  # Display the predicted label
        ax[i, j].axis("off")
        idx += 1

plt.tight_layout()
plt.suptitle("Test Dataset Predictions", fontsize=20)
plt.subplots_adjust(top=0.88)  # Adjust the top spacing so the title doesn't overlap the subplots
plt.show()

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend()
plt.figure()
plt.show()